In [11]:
pip install tensorflow numpy nltk


Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
# Turn off GPU and suppress TensorFlow logs
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Step 1: Define training data (Q&A pairs)
pairs = [
    ["hi", "hello"],
    ["how are you?", "i am fine"],
    ["what is your name?", "i am a chatbot"],
    ["bye", "goodbye"]
]

questions, answers = zip(*pairs)
questions = list(questions)
answers = ["<start> " + a + " <end>" for a in answers]

# Step 2: Tokenize and pad sequences
tokenizer = Tokenizer(filters='', oov_token="<OOV>")
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index) + 1

question_seqs = tokenizer.texts_to_sequences(questions)
answer_seqs = tokenizer.texts_to_sequences(answers)
max_len = max(max(len(q) for q in question_seqs), max(len(a) for a in answer_seqs))

question_seqs = pad_sequences(question_seqs, maxlen=max_len, padding='post')
answer_seqs = pad_sequences(answer_seqs, maxlen=max_len, padding='post')

# Prepare decoder target data
decoder_target = np.zeros_like(answer_seqs)
decoder_target[:, :-1] = answer_seqs[:, 1:]
decoder_target = np.expand_dims(decoder_target, -1)

# Step 3: Build the model
embedding_dim = 64
latent_dim = 128

encoder_inputs = Input(shape=(None,))
decoder_inputs = Input(shape=(None,))
embedding_layer = Embedding(VOCAB_SIZE, embedding_dim)

enc_emb = embedding_layer(encoder_inputs)
dec_emb = embedding_layer(decoder_inputs)

_, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit([question_seqs, answer_seqs], decoder_target, epochs=300, verbose=0)

# Step 4: Define inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = embedding_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2, state_h2, state_c2]
)

# Step 5: Define response generation function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq, verbose=0)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<start>']
    
    decoded_sentence = ''
    for _ in range(max_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '')
        
        if sampled_word == '<end>' or sampled_word == '':
            break
        decoded_sentence += ' ' + sampled_word
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence.strip()

# Step 6: Start interactive chatbot loop
print("Chatbot is ready! Type 'exit' to quit.\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Bot: Goodbye!")
        break
    input_seq = tokenizer.texts_to_sequences([user_input])
    input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')
    response = decode_sequence(input_seq)
    print("Bot:", response)


Chatbot is ready! Type 'exit' to quit.



You:  hai


Bot: goodbye


You:  hello


Bot: goodbye


You:  what is your name


Bot: i am a chatbot
